# Fine-Tuning LLaMA with LoRA for Math Word Problems

This notebook demonstrates how to fine-tune the LLaMA-3.2-1B model using LoRA (Low-Rank Adaptation) on the Orca Math Word Problems dataset. We support multiple modes (Torch, 8-bit, 16-bit) and include evaluation metrics like perplexity.

## Setup and Imports

In [1]:
# !pip install -r requirements.txt

In [ ]:
import math
import os
import random
import shutil
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from utils_lora import generate_and_print

from concrete.ml.torch.lora import LoraTrainer

## Configuration Parameters

Define the training mode and other parameters here. Modify these values as needed before running the notebook.

In [ ]:
# Training mode: 'torch', '7bit', or '16bit'
MODE = "7bit"

# Number of steps between evaluations
EVAL_STEPS = 100

# Set to True to force CPU
FORCE_CPU = False

# File paths based on mode
mode_str = MODE
EVAL_RESPONSES_FILE = f"eval_generated_responses_{mode_str}.txt"
TRAIN_LOG_FILE = f"training_log_{mode_str}.txt"
SAVE_PATH = Path(f"deployment/llama_lora_finetuned_{mode_str}")

print(f"Mode: {MODE}, Eval Steps: {EVAL_STEPS}, Force CPU: {FORCE_CPU}")

## Device and Seed Configuration

Set up the device (CPU/GPU/MPS) and random seeds for reproducibility.

In [ ]:
def get_device():
    if FORCE_CPU:
        return "cpu"
    if torch.cuda.is_available():
        return "cuda"
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
        return "mps"
    return "cpu"


def set_seed(seed):
    """Set random seeds for reproducibility.

    Args:
        seed (int): The random seed to use
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.use_deterministic_algorithms(True, warn_only=True)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False


SEED = 0
DEVICE = get_device()
set_seed(SEED)
print(f"Using device: {DEVICE}")

## Load Model and Tokenizer

Load the LLaMA model and tokenizer, and test the base model output.

In [ ]:
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(DEVICE)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id
for p in model.parameters():
    p.requires_grad = False

PROMPT = "When you multiply a number by 7, it becomes 98. What is that number?\n"
_ = generate_and_print(PROMPT, model, tokenizer, seed=SEED)

## LoRA Configuration

Set up LoRA parameters and apply them to the model.

In [6]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.01,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear",
)
peft_model = get_peft_model(model, peft_config)

## Dataset Preprocessing

Load the dataset, filter by length, and preprocess for training.

In [ ]:
MAX_LENGTH = 64
raw_dataset = load_dataset("microsoft/orca-math-word-problems-200k", split="train")


def length_filter(example):
    q_len = len(tokenizer(example["question"], add_special_tokens=False)["input_ids"])
    a_len = len(tokenizer(example["answer"], add_special_tokens=False)["input_ids"])
    return (q_len + a_len + 1) <= MAX_LENGTH


filtered_dataset = raw_dataset.filter(length_filter)


def get_lengths(example):
    q_len = len(tokenizer(example["question"], add_special_tokens=False)["input_ids"])
    a_len = len(tokenizer(example["answer"], add_special_tokens=False)["input_ids"])
    total_len = q_len + a_len + 1
    return {"q_len": q_len, "a_len": a_len, "total_len": total_len}


lengths = filtered_dataset.map(get_lengths)
q_lengths = [x["q_len"] for x in lengths]
a_lengths = [x["a_len"] for x in lengths]
total_lengths = [x["total_len"] for x in lengths]

print("\nLength Distribution Statistics:")
print(f"Original dataset size: {len(raw_dataset):,}")
print(f"Filtered dataset size: {len(filtered_dataset):,}")
print(f"Percentage kept: {100 * len(filtered_dataset)/len(raw_dataset):.1f}%\n")
print("Question lengths: ")
print(f"  Min: {min(q_lengths)}, Max: {max(q_lengths)}")
print(f"  Mean: {sum(q_lengths)/len(q_lengths):.1f}")
print(f"  Median: {sorted(q_lengths)[len(q_lengths)//2]}")
print("\nAnswer lengths:")
print(f"  Min: {min(a_lengths)}, Max: {max(a_lengths)}")
print(f"  Mean: {sum(a_lengths)/len(a_lengths):.1f}")
print(f"  Median: {sorted(a_lengths)[len(a_lengths)//2]}")
print("\nTotal lengths (including newline):")
print(f"  Min: {min(total_lengths)}, Max: {max(total_lengths)}")
print(f"  Mean: {sum(total_lengths)/len(total_lengths):.1f}")
print(f"  Median: {sorted(total_lengths)[len(total_lengths)//2]}\n")


def process_example(example):
    """Tokenize a question-answer pair and prepare labels for training.

    Args:
        example (dict): Dictionary with 'question' and 'answer' strings
    Returns:
        dict: Processed tokens with masked labels for the question portion
    """
    question = example["question"].strip()
    answer = example["answer"].strip()
    tokens = tokenizer(
        question + "\n" + answer,
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH,
    )
    question_length = len(tokenizer(question, add_special_tokens=False)["input_ids"]) + 1
    labels = tokens["input_ids"].copy()
    for i in range(question_length):
        if i < len(labels):
            labels[i] = -100
    tokens["labels"] = labels
    return tokens


tokenized_dataset = filtered_dataset.map(
    process_example,
    batched=False,
    remove_columns=filtered_dataset.column_names,
)

tokenized = tokenized_dataset.train_test_split(test_size=0.05, seed=SEED, shuffle=True)
train_dataset, test_dataset = tokenized["train"], tokenized["test"]
print(f"Train samples: {len(train_dataset)}, Test samples: {len(test_dataset)}")

## Data Collator

Define a custom data collator to handle padding while preserving label masking.

In [8]:
class DataCollator:
    def __init__(self, tokenizer):
        self.pad_id = tokenizer.pad_token_id

    def __call__(self, examples):
        inputs, attention_masks, labels = [], [], []
        max_length = max(len(example["input_ids"]) for example in examples)
        for example in examples:
            inputs.append(example["input_ids"])
            attention_masks.append(example["attention_mask"])
            labels.append(example["labels"])

        def pad(sequences, value):
            return [x + [value] * (max_length - len(x)) for x in sequences]

        return {
            "input_ids": torch.tensor(pad(inputs, self.pad_id)),
            "attention_mask": torch.tensor(pad(attention_masks, 0)),
            "labels": torch.tensor(pad(labels, -100)),
        }


collator = DataCollator(tokenizer)

## Training Arguments

Configure the training hyperparameters.

In [9]:
training_args = TrainingArguments(
    output_dir="./checkpoints",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    save_total_limit=1,
    use_cpu=True,
    learning_rate=2e-4,
    lr_scheduler_type="linear",
    seed=SEED,
    data_seed=SEED,
    warmup_steps=10,
    weight_decay=0.01,
    prediction_loss_only=True,
    report_to="none",
)

## Loss and Evaluation Metrics

Define the loss function and evaluation metric (perplexity).

In [10]:
def causal_lm_loss(logits, labels):
    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    return F.cross_entropy(
        shift_logits.view(-1, shift_logits.size(-1)),
        shift_labels.view(-1),
        ignore_index=-100,
        reduction="mean",
    )


def metric_fn(model, dataloader):
    model.eval()
    model.to(DEVICE)
    total_loss, total_tokens, results = 0.0, 0, []
    response = generate_and_print(PROMPT, model, tokenizer, seed=SEED)
    if response:
        results.append({"prompt": PROMPT, "response": response})

    for batch in tqdm(dataloader, desc="Evaluating", leave=False):
        with torch.no_grad():
            input_ids = batch["input_ids"].to(DEVICE)
            batch_labels = batch["labels"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            outputs = model(input_ids, attention_mask=attention_mask).logits
            valid = batch_labels[..., 1:] != -100
            loss = F.cross_entropy(
                outputs[..., :-1, :].contiguous().view(-1, outputs.size(-1)),
                batch_labels[..., 1:].contiguous().view(-1),
                ignore_index=-100,
                reduction="sum",
            )
        total_loss += loss.item()
        total_tokens += valid.sum().item()
    perplexity = math.exp(total_loss / total_tokens) if total_tokens > 0 else float("inf")

    with open(EVAL_RESPONSES_FILE, "a", encoding="utf-8") as f:
        f.write(f"Perplexity: {perplexity:.2f}\n")
        for i, r in enumerate(results):
            f.write(
                f"== Generation {i+1} ==\nPrompt:\n{r['prompt']}\n\nResponse:\n{r['response']}\n"
            )
            f.write("=" * 40 + "\n")

    return {"perplexity": perplexity}

## Trainer Setup

Initialize the Hugging Face Trainer to set up the optimizer and scheduler.

In [11]:
hf_trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=collator,
)
train_dl = hf_trainer.get_train_dataloader()
hf_trainer.create_optimizer_and_scheduler(len(train_dl) * training_args.num_train_epochs)
optimizer, lr_scheduler = hf_trainer.optimizer, hf_trainer.lr_scheduler

## Calibration and Evaluation Data

Prepare dummy calibration data and the evaluation DataLoader.

In [12]:
def get_inputset():
    return {
        "input_ids": torch.randint(0, tokenizer.vocab_size, (4, MAX_LENGTH)),
        "attention_mask": torch.ones((4, MAX_LENGTH), dtype=torch.long),
        "labels": torch.randint(0, tokenizer.vocab_size, (4, MAX_LENGTH)),
    }


eval_dl = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collator)

## Model Initialization Function

Add a function to initialize models with consistent weights for our comparison experiments.

In [13]:
def initialize_model(seed=SEED):
    """Initialize the model with consistent weights."""
    set_seed(seed)

    model = AutoModelForCausalLM.from_pretrained(model_name).to(DEVICE)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id
    for p in model.parameters():
        p.requires_grad = False

    peft_config = LoraConfig(
        r=8,
        lora_alpha=32,
        lora_dropout=0.01,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules="all-linear",
    )
    peft_model = get_peft_model(model, peft_config)
    peft_model = peft_model.to(DEVICE)
    return peft_model


def setup_trainer(model, steps=EVAL_STEPS):
    """Set up the trainer, optimizer and scheduler."""
    hf_trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=collator,
    )
    train_dl = hf_trainer.get_train_dataloader()
    hf_trainer.create_optimizer_and_scheduler(len(train_dl) * training_args.num_train_epochs)
    optimizer, lr_scheduler = hf_trainer.optimizer, hf_trainer.lr_scheduler

    lora_trainer = LoraTrainer(
        model=model,
        optimizer=optimizer,
        loss_fn=causal_lm_loss,
        lr_scheduler=lr_scheduler,
        training_args=vars(training_args),
        n_layers_to_skip_for_backprop=3,
        eval_loader=eval_dl,
        eval_metric_fn=metric_fn,
        logging_steps=1,
        eval_steps=steps,
        train_log_path=TRAIN_LOG_FILE,
    )

    if MODE != "torch":
        bits = 7 if MODE == "7bit" else 16
        lora_trainer.compile(get_inputset(), n_bits=bits)

    return lora_trainer, train_dl


def extract_lora_weights(model):
    """Extract LoRA weights from model for comparison."""
    weights = {}
    for name, param in model.named_parameters():
        if "lora_" in name and param.requires_grad:
            weights[name] = param.detach().cpu().numpy().copy()
    return weights


# Select only the first 5 batches for initial comparison
def get_limited_batches(dataloader, num_batches=5):
    """Get a limited number of batches from dataloader."""
    limited_batches = []
    for i, batch in enumerate(dataloader):
        if i < num_batches:
            limited_batches.append(batch)
        else:
            break
    return limited_batches

## Experiment: Comparing fhe="disable" vs fhe="execute"

Train two identical models for 5 steps using the quantized clear model (disable mode) and compare against FHE fine-tuning (execute mode)

In [ ]:
# Original model trainer for reference
lora_trainer = LoraTrainer(
    model=peft_model,
    optimizer=optimizer,
    loss_fn=causal_lm_loss,
    lr_scheduler=lr_scheduler,
    training_args=vars(training_args),
    n_layers_to_skip_for_backprop=3,
    eval_loader=eval_dl,
    eval_metric_fn=None,
    logging_steps=1,
    eval_steps=EVAL_STEPS,
    train_log_path=TRAIN_LOG_FILE,
)

# Clear the existing file for our experiments
with open(EVAL_RESPONSES_FILE, "w", encoding="utf-8") as f:
    f.write("=== Experiment: disable vs simulate ===\n")

In [ ]:
print("Initializing first model (fhe=disable)...")
model_disable = initialize_model()
initial_weights_disable = extract_lora_weights(model_disable)

trainer_disable, train_dl = setup_trainer(model_disable)

# Evaluate pre-training
print("Evaluating pre-training model...")
model_disable.eval()
pre_training_metrics = metric_fn(model_disable, eval_dl)
print(f"Pre-training perplexity: {pre_training_metrics['perplexity']:.2f}")

# Get limited batches for consistent training
limited_batches = get_limited_batches(train_dl, 5)

# Train with fhe=disable for 5 steps
print("\nTraining with fhe=disable for 5 steps...")
trainer_disable.train(limited_batches, fhe="disable", device=DEVICE)
losses_disable = trainer_disable.get_training_losses()

# Store LoRA weights
weights_disable = extract_lora_weights(model_disable)

In [ ]:
print("\nInitializing second model (fhe=execute)...")
model_execute = initialize_model()
trainer_execute, _ = setup_trainer(model_execute)

# Verify weights are the same before training
print("Verifying initial weights are identical...")
initial_weights_execute = extract_lora_weights(model_execute)
weights_match = True
for name in initial_weights_disable:
    if not np.allclose(initial_weights_disable[name], initial_weights_execute[name]):
        print(f"Weight mismatch in {name}")
        weights_match = False
if weights_match:
    print("Initial weights match between models.")

# Train with fhe=execute for 5 steps
print("\nTraining with fhe=execute for 5 steps...")
trainer_execute.train(limited_batches, fhe="disable", device=DEVICE)
losses_execute = trainer_execute.get_training_losses()

# Compare LoRA weights after training
weights_execute = extract_lora_weights(model_execute)

In [ ]:
# Create figure
plt.figure(figsize=(12, 8))

# Get x positions (steps)
steps = np.arange(len(losses_disable))

# Plot lines with enhanced visibility
plt.plot(steps, losses_disable, label="Disable Loss", color="blue", linewidth=2.5, alpha=0.7)
plt.plot(
    steps,
    losses_execute,
    label="Execute Loss",
    color="red",
    linewidth=2.5,
    alpha=0.7,
    linestyle="--",
)

# Add connecting lines to highlight differences
for i in range(len(losses_disable)):
    plt.plot(
        [i, i], [losses_disable[i], losses_execute[i]], color="purple", alpha=0.5, linewidth=1.5
    )

# Add points with distinct markers and sizes
plt.scatter(steps, losses_disable, color="blue", s=100, zorder=5, label="_nolegend_")
plt.scatter(steps, losses_execute, color="red", marker="s", s=100, zorder=5, label="_nolegend_")

plt.xlabel("Step", fontsize=12)
plt.ylabel("Loss", fontsize=12)
plt.title("Comparison of Losses with Highlighted Differences", fontsize=14)
plt.legend(fontsize=12, loc="upper right")
plt.grid(True, linestyle=":", alpha=0.4)

# Set x-ticks to be at each step
plt.xticks(steps)

plt.tight_layout()
plt.show()

## Continue Training for Full Convergence

After validating the models, we'll continue training on the remaining batches (500-5 = 495 steps).

In [ ]:
print("\n=== Continuing Training for Full Convergence ===")
# We'll use the model_disable for further training
model_final = initialize_model()
trainer_final, full_train_dl = setup_trainer(model_final)

# Get remaining batches (skip the first 5 we already used)
remaining_batches = []
for i, batch in enumerate(full_train_dl):
    if i >= 5 and i < 500:
        remaining_batches.append(batch)
    elif i >= 500:
        break

print(f"Training on {len(remaining_batches)} additional batches...")
trainer_final.train(remaining_batches, fhe="disable", device=DEVICE)

# Final evaluation
print("\nEvaluating final model after full training...")
model_final.eval()
metrics_final = metric_fn(model_final, eval_dl)
print(f"Final perplexity after extended training: {metrics_final['perplexity']:.2f}")
print(
    f"Improvement from initial model: {pre_training_metrics['perplexity'] - metrics_final['perplexity']:.2f}"
)

## Save the Model

Save the fine-tuned model to disk.

In [ ]:
if SAVE_PATH.is_dir() and any(SAVE_PATH.iterdir()):
    shutil.rmtree(SAVE_PATH)
trainer_final.save_and_clear_private_info(SAVE_PATH)
print("Model saved to:", SAVE_PATH)